Limit Chase Order Agent

In [ ]:
import DEMO_base_fxns as base

# Pair and bid price
perp_name = 'BTC' + 'USDT'
bid_price = 94000

# Parameters --------------------------------------------------------------------------------------------------------------
collateral = 10000
sL_perc = 0.75
tP_perc = 500
leverage=2
#--------------------------------------------------------------------------------------------------------------------------
# Main Script
try:
    # Helper Functions------------------------------------------------------------------------------------------------------------
    def check_balance(perp_name):
         # Don't buy if we already got coins
        info_balance = base.session.get_wallet_balance(accountType="UNIFIED",
                                                       coin = perp_name.split('USDT')[0])
        info_usdCoinBal = info_balance['result']['list'][0]['coin'][0]['usdValue']
        if info_usdCoinBal == '':
            existingperpBalUSD = 0
        else:
            existingperpBalUSD = float(info_usdCoinBal)
        if existingperpBalUSD >= 30:
            return f'REJECT:Already Bought ${existingperpBalUSD} of {perp_name}.'
        else:
            return ''
    def position_exists(perp_name): # Check if position exists
        base.session.get_positions(
            category="linear",
            settleCoin="USDT"
        )['result']['list'][0]['symbol']

        open_positions = base.session.get_positions(
            category="linear",
            settleCoin="USDT")['result']['list']

        for op in open_positions:
            if op['symbol'] == perp_name:
                return True
        return False    
    def get_order_qty(perp_price,params,usdt_amount): # REQUIRED Order Setup: Quantity, need to follow certain rules/format
        qtyStepCoin = params['perp_info']['qtyStepCoin'] #pre-loaded from API
        coin_precision_info = params['perp_info']['coin_precision_info'] #pre-loaded from API
        if '.' in coin_precision_info:
            coin_precision_info = len(coin_precision_info.split('.')[1])
            theo_coin_qty = round(usdt_amount/float(perp_price),coin_precision_info) # desired quantity
            actual_coin_qty = round(theo_coin_qty - (theo_coin_qty%qtyStepCoin),coin_precision_info) 
        elif coin_precision_info == '1':
            theo_coin_qty = round(usdt_amount/float(perp_price)) # desired quantity
            actual_coin_qty = round(theo_coin_qty - (theo_coin_qty%qtyStepCoin))
        else:
            coin_precision_info = -1*len(coin_precision_info)
            theo_coin_qty = round(usdt_amount/float(perp_price),coin_precision_info) # desired quantity
            actual_coin_qty = round(theo_coin_qty - (theo_coin_qty%qtyStepCoin),coin_precision_info)
        return str(actual_coin_qty) 
    def buy_limit_chase_order(perp_price,usdt_amount,params):
        try:
            perp_name = params['perp_name']
            # Check if we have enough usdt
            balanceCheck = check_balance(perp_name=perp_name)
            if 'REJECT' in balanceCheck:
                return balanceCheck
            # Place Order
            response = base.session.place_order(
                category="linear",
                symbol=perp_name,
                side="Buy",
                orderType="Limit",
                qty=get_order_qty(perp_price=perp_price,params=params,
                                  usdt_amount=usdt_amount),
                price = perp_price,
                timeInForce="PostOnly")
            
            if response['retMsg']=='OK':
                # Post-Processing
                oid = response['result']['orderId']
                size = round(usdt_amount,2)
                ts_ms = response['time'] # in msec
                ts_dt = base.datetime.fromtimestamp(ts_ms/1000)
                formatted_time = ts_dt.strftime('%Y-%m-%d %H:%M:%S')
                logger.info(f'Limit Bid ${str(size)} of {perp_name} @ {perp_price} @ {ts_dt}')

                return True
        except Exception as e:
            # Log the exception
            if 'ab not enough for new order' in str(e):
                # base.logging.error(f"REJECT: Can\'t make the min. order...")
                print(f'\nErrored!{e}')
                return 'REJECT: Can\'t make the min. order...used up all USDT'
            else:
                return f'Unhandled REJECT/ERROR 2: {e}'   
    def buy_limit_chase_order_manage(perp_price,params):

        # Check if 1st Limit Order was successful(position exists)
        if position_exists(params['perp_name']):
            params['limit-chase-mode'] = False
            return params

        open_orders = base.session.get_open_orders(category="linear", settleCoin="USDT")['result']['list']
        limit_ord_count = sum(1 for oo in open_orders if oo['symbol'] == perp_name)
        if limit_ord_count > 1:
            logger.info(f'MULTIPLE LIMIT ORDERS....stopping WebSocket.')
            OnKeyboardInterrupt()
            # return f'Unhandled REJECT/ERROR 1'
            
        elif limit_ord_count == 0: #assume position_exists returned false
             # This case is when 1st post order got canceled
            buy_limit_chase_order(perp_price=perp_price,
                                usdt_amount=params['collateral'][params['attempt_num']],
                                params=params)

        elif limit_ord_count == 1:
            open_oid = oo['orderId']
            ooPrice = float(oo['price'])
            # Update limit order closer to market price
            if (markPrice>=1.0005*ooPrice) or (markPrice<=(1-0.0005)*markPrice):
                try:
                    base.session.cancel_order(
                        category="linear",
                        symbol=perp_name,
                        orderId=oo['orderId'])
                except Exception as e:
                    if 'too late to cancel' in e:
                        print(e)
                
        return
    def sell_all_perp(perp_name):
        symbol_info = base.session.get_instruments_info(category='perp', symbol=perp_name)
        info_balance = base.session.get_wallet_balance(accountType="UNIFIED",
                                               coin = perp_name.split('USDT')[0])
        coin_qty = float(info_balance['result']['list'][0]['coin'][0]['walletBalance'])
        coin_usd_value = round(float(info_balance['result']['list'][0]['coin'][0]['usdValue']))

        if coin_usd_value > 5:
            symbol_info = base.session.get_instruments_info(category='perp', symbol=perp_name)
            coin_precision_info = symbol_info['result']['list'][0]['lotSizeFilter']['basePrecision']
            if '.' in coin_precision_info:
                # coin_precision_info = len(coin_precision_info.split('.')[1])
                # coins_qty_to_sell = round(coin_qty,coin_precision_info)
                coin_precision_info = float(coin_precision_info)
                coins_qty_to_sell = base.math.floor(coin_qty/coin_precision_info) * coin_precision_info
                coins_qty_str = str(coins_qty_to_sell).split('.')
                coin_precision_len = len(str(coin_precision_info).split('.')[-1])
                coins_qty_to_sell = float(coins_qty_str[0] + '.' + coins_qty_str[1][:coin_precision_len])
            elif coin_precision_info=='1':
                coins_qty_to_sell = base.math.floor(coin_qty)
            else:
                coin_precision_info = -1*len(coin_precision_info)
                coins_qty_to_sell = round(coin_qty,coin_precision_info)


            response = base.session.place_order(
                category="linear",
                symbol=perp_name,
                side="Sell",
                orderType="Market",
                qty=str(coins_qty_to_sell),
                marketUnit = 'baseCoin')
            
            
            logger.info(f'Sold {str(coins_qty_to_sell)} {perp_name} for {coin_usd_value}')
            # logger.info('Position was already closed.')
        else:
            print('Position was already closed.')
    #-----------------------------------------------------------------------------------------------------------------------------
    # Strategy: Long on reclaim
    def reclaimLong_strat(perp_price,parameters,date_time):
        #--------------------------------------------------------------------------------------------------------------------------
        # Load in parameters
        p = parameters
        anum = p['attempt_num']
        key_level = p['key_level']
        longedBool = p['longedBool']
        SL_perc = p['stop_loss_perc']/100
        TP_perc = p['take_profit_perc']/100
        #-----------------------------------------------------------------------------------------------------------------------------
        # if a stop_loss_price/take_profit_price exists, is set when we enter otherwise None
        SL_price = None
        TP_price = None
        if p['stop_loss_price'].get(anum+1) != None: 
            SL_price = p['stop_loss_price'][anum+1]
        if p['take_profit_price'].get(anum+1) != None: 
            TP_price = p['take_profit_price'][anum+1]

        # TRADE MANAGEMENT ##**##**##**##**##**##**##**##**##**##**##**##**##**##**##**##**##**##**##
        # 1st Condition Check: If we're below key level and no long open. 
        if perp_price < key_level and longedBool==False and p['condBelowSupport']==False:
            p['condBelowSupport'] = True
            logger.info(f'Below key Lvl : {date_time} @ {perp_price} , Anum:{anum}')  
            logger.info(f'Looking for reclaim.....')

        #-----------------------------------------------------------------------------------------------------------------------------
        # LONG on reclaim of key level.
        elif perp_price >= key_level and p['condBelowSupport']==True and p['longedBool']==False:
            # logger.info('we buyyinnggg')
            p['condBelowSupport'] = False
            p['longedBool'] = True
        
            # Record time and price of entry_price for analysis(All or nothing parameter flag)
            if anum+1 not in p['entries']:
                p['entries'][anum+1] = {}
                p['entries'][anum+1][date_time] = perp_price
            else:
                p['entries'][anum+1][date_time] = perp_price

            # EDITED: using  in SL/TP
            p['stop_loss_price'][anum+1] = perp_price*(1-SL_perc)
            p['take_profit_price'][anum+1] = perp_price*(1+TP_perc)

            # ***Ensure there isn't already an open order incase of thread collision
            # PLACE ORDER
            buy_msg = buy_limit_chase_order(perp_price = perp_price,
                                            usdt_amount=p['collateral'][anum],
                                            target_bid=key_level, # will have to change to Market for aggressive reclaim long
                                            TP_price=p['take_profit_price'][anum+1],
                                            SL_price=p['stop_loss_price'][anum+1],
                                            perp_name=params['perp_name'],
                                            params=params)
            
            if buy_msg==True:
                params['limit-chase-mode'] = True # Flip this on to manage limit-chase-mode
                logger.info(f'Reclaimed:{key_level}|Limit Bid@{perp_price}|SL:{p['stop_loss_price'][anum+1]}|TP:{p['take_profit_price'][anum+1]}')
            elif 'REJECT:Already Bought' not in buy_msg:
                # Potential Issue(Ideally shouldn't have placed order in buy_order fxn, then break WS) 
                logger.info(f'UNKNOWN ERROR \n {buy_msg}....stopping WebSocket.')
                OnKeyboardInterrupt()
                return f'Unhandled REJECT/ERROR 1:{buy_msg}'
        #-----------------------------------------------------------------------------------------------------------------------------
        # STOP LOSS exit
        elif longedBool==True and SL_price != None:
            if perp_price < SL_price:
                # Close position
                try:
                    sell_all_perp(perp_name)
                except Exception as e:
                    if 'InvalidRequestError: The number of contracts exceeds minimum limit allowed' in str(e):
                        logger.info('No position closed. Likely already closed by Stop Loss fxn.')
                    else:
                        logger.info(f'ERROR in selling:{str(e)}')


                # reset conditions to long, make sure you don't reenter forever
                longedBool = False
                p['longedBool']=False
                p['condBelowSupport'] = True
                # calculate loss
                current_collateral = p['collateral'][anum]
                loss,pnl_perc = base.tradeStats.perp_pnl(entry=list(p['entries'][anum+1].values())[0],
                                                    exit=perp_price,
                                                    leverage=1,
                                                    side='Long',
                                                    collateral=current_collateral)
                # update/reset params
                p['losses'][anum+1] = loss
                p['profits'][anum+1] = 0
                p['collateral'][anum+1] = current_collateral + loss #loss is negative

                # Record time and price of exit_price for analysis
                if anum+1 not in p['exits']:
                    p['exits'][anum+1] = {}
                    p['exits'][anum+1][date_time] = perp_price
                else:
                    p['exits'][anum+1][date_time] = perp_price

                # p['stop_loss_price'][anum+2] = None  # hard reset/prolly not even needed tbh
                p['attempt_num'] = p['attempt_num'] + 1 # End of trade/attempt

                # notify
                # tgNotifs.stop_loss_msg(perp_price,loss,p)
                # print(f'SL_{perp_price} @ {date_time}|Loss:{loss}|{pnl_perc}% \n')
                logger.info(f'Stopped at {perp_price} @ {date_time}| Loss:{loss}||{pnl_perc}%')
                logger.info(f'---- Collateral:{round(current_collateral,2)} -> {round(p['collateral'][anum+1],2)} ----')

        #-----------------------------------------------------------------------------------------------------------------------------
        # TAKE PROFIT exit
        if TP_price != None:
            if perp_price > TP_price:
                # Close position
                try:
                    sell_all_perp(perp_name)
                except Exception as e:
                    if 'InvalidRequestError: The number of contracts exceeds minimum limit allowed' in str(e):
                        logger.info('No position closed. Likely already closed by Stop Loss fxn.')
                if longedBool:
                    # logger.info('block 4')
                    p['longedBool']=False
                    p['condBelowSupport'] = False # should already be false
                    # calculate profit
                    current_collateral = p['collateral'][anum]
                    profit,pnl_perc = base.tradeStats.perp_pnl(entry=list(p['entries'][anum+1].values())[0],
                                                            exit=perp_price,
                                                            leverage=1,
                                                            side='Long',
                                                            collateral=current_collateral)
                    # update params
                    p['losses'][anum+1] = 0
                    p['profits'][anum+1] = profit
                    p['collateral'][anum+1] = current_collateral + profit 
                    # Record time and price of exit_price for analysis
                    if anum+1 not in p['exits']:
                        p['exits'][anum+1] = {}
                        p['exits'][anum+1][date_time] = perp_price
                    else:
                        p['exits'][anum+1][date_time] = perp_price
                    
                    # p['take_profit_price'][anum+2] = None  # hard reset/prolly not even needed tbh
                    p['attempt_num'] = p['attempt_num'] + 1 # End of trade/attempt

                    # think about when we hit a stop loss and need to reconsider parameters/conditions for long
                        #regardless of long, always calculate EMA condition 
                    print(f'TP@{perp_price} @ {date_time}|P:{profit}|{pnl_perc}% \n')
                    logger.info(f'TP@{perp_price} @ {date_time}|+{profit}|{pnl_perc}%')
                    logger.info(f'---- Collateral:{current_collateral} -> {round(p['collateral'][anum+1],2)} ----')
                    
                    #Stop WS
                    OnKeyboardInterrupt()
                    # notify
                    # tgNotifs.stop_loss_msg(perp_price,loss,p)
        return p
    #-----------------------------------------------------------------------------------------------------------------------------
    # Handle buy/wsErrors
    def OnKeyboardInterrupt():
        if ws.is_connected():
            ws.exit()
        endTimeTS = int(base.time.time())
        endTime = base.datetime.fromtimestamp(endTimeTS).strftime("[%m-%d-%y]%H%M")
        finalLogFileName = f'{startLogFileName}{endTime}'
        finalDfFileName = ws_df_path.split('.')[0] + endTime + '.'+ ws_df_path.split('.')[1]
        finalParamsFileName = startParamsFileName.split('.')[0] + endTime + '.'+ startParamsFileName.split('.')[1]
        #Rename log file and ws feather file with endTime
        base.os.rename(f'{log_filePath}/{startLogFileName}.log', f'{log_filePath}/{finalLogFileName}.log')
        base.os.rename(ws_df_path,finalDfFileName)
        if prev_attempt>0:
            base.os.rename(f'{params_filePath}/{startParamsFileName}',f'{params_filePath}/{finalParamsFileName}')
        print(f"\n---Closing {perp_name} data to log")
        print(f"---Closing {perp_name} trade log:{log_filePath}/{finalLogFileName}.log ---")
        print((f"-------\nClosing {perp_name} data to:{finalDfFileName }"))
        logger.info(f"------\nClosing {perp_name} data to:{finalDfFileName}")
        exit()
    def handle_error(error):
        base.logging.error(f"WebSocket encountered error: {error}")
    def reconnect_websocket():
        global ws
        try:
            ws = base.WebSocket(testnet=False, channel_type="perp")
            ws.ticker_stream(symbol=perp_name, callback=handle_message_wrapper(logger))
            logger.info("WebSocket reconnected successfully.")
        except Exception as e:
            logger.error(f"Reconnection failed: {e}")
    def handle_disconnection():
        logger.warning("WebSocket disconnected, attempting to reconnect...")
        backoff_time = 2  # Start with 2 seconds
        max_retries = 5   # Maximum number of retries

        for attempt in range(max_retries):
            try:
                reconnect_websocket()
                if ws.is_connected():
                    logger.info("WebSocket reconnected after disconnection.")
                    return True
            except Exception as e:
                logger.error(f"Reconnection attempt {attempt + 1} failed: {e}")
            
            # Exponential backoff
            base.time.sleep(backoff_time)
            backoff_time *= 2

        logger.error("WebSocket could not reconnect after multiple attempts.")
        return False
    #-----------------------------------------------------------------------------------------------------------------------------
    def intialize_system_variables():
        global wsIndexPs,wsMarksPs,DEMO_path,startTimeTS,time_index,startTimeTS,time_index,startTime,startDfFileName,ws_df_path
        global startLogFileName,log_filePath
        global params_filePath,startParamsFileName,params_to_write
        DEMO_path = '/Users/DEMO'
        startTimeTS = int(base.time.time())
        time_index = int(base.time.time())
        startTime = base.datetime.fromtimestamp(startTimeTS).strftime("[%m-%d-%y]%H%M")
        startLogFileName = f'{perp_name}_tradeLog_{startTime}_'
        startDfFileName = f'{perp_name}_{startTime}_.feather'
        startParamsFileName = f'{perp_name}_params_{startTime}_.json'
        ws_df_path = f'{DEMO_path}/ws_data/{startDfFileName}'
        log_filePath = f'{DEMO_path}/tradeLogs'
        params_filePath =  f'{DEMO_path}/params_logs'
        params_to_write = ['collateral','entries','exits','losses','profits','stop_loss_price','take_profit_price']
        wsMarksPs= {}
        wsIndexPs= {}
    def logging():
        global logger
        base.logging.basicConfig(filename=f'{log_filePath}/{startLogFileName}.log',
                                filemode='w',
                                format="%(asctime)s %(message)s",
                                datefmt='%m-%d-%y %I:%M:%S %p|',
                                level=base.logging.ERROR,
                                force=True)
        logger = base.logging.getLogger()
        logger.setLevel(base.logging.INFO)
        logger.info(f"Longing perp={perp_name}|KeyLevel={bid_price}|Collateral={collateral}|stopLossPerc={sL_perc}|takeProfitPerc={tP_perc}")
    #-----------------------------------------------------------------------------------------------------------------------------
     # Websocket Data Management: Receive msg with markPrice -> feed markPrice to strategy
    def handle_message_wrapper(logger):
        def handle_message(msg):
            #-----------------------------------------------------------------------------------------------------------------------------
            # Websocket data Parse
            # print(msg)
            data = msg['data']
            symbol = data['symbol']

            markPrice = data['markPrice'] # perp price
            markPrice = float(markPrice)
            timeStamp = msg['ts'] / 1000 # Convert msec to seconds
            dtime = base.datetime.fromtimestamp(timeStamp)
            formatted_datetime = dtime.strftime('%m/%d/%y %I:%M:%S %p')
            #-----------------------------------------------------------------------------------------------------------------------------
            # Mark and Index Price Data store
            wsMarksPs[formatted_datetime] = markPrice
            # wsIndexPs[formatted_datetime] = indexPrice
            
            #-----------------------------------------------------------------------------------------------------------------------------
            if params['limit-chase-mode']==True:
                buy_limit_chase_order_manage(markPrice,params)
            # Run strategy
            params_update = reclaimLong_strat(perp_price=markPrice,
                                            parameters=params,
                                            date_time=formatted_datetime)
            if 'ERROR' in params_update:
                OnKeyboardInterrupt()
            else:
                # Update parameters from strategy
                for parameter in params.keys():     
                    params[parameter] = params_update[parameter]
        return handle_message
    #-----------------------------------------------------------------------------------------------------------------------------
    # BEGIN/SETUP: Parameters, system variables,
    params = base.intialize_perp_params(perp_name=perp_name,
                            key_level=bid_price,
                            collat=collateral,
                            leverage=leverage,
                            take_profit_perc=tP_perc,
                            stop_loss_perc=sL_perc) 
    symbol_info = base.session.get_instruments_info(category='perp', symbol=perp_name)
    params['perp_info']['qtyStepCoin'] = float(symbol_info['result']['list'][0]['priceFilter']['tickSize'])  #min coin increment
    params['perp_info']['coin_precision_info'] = float(symbol_info['result']['list'][0]['lotSizeFilter']['tickSize'])  #min coin increment
    intialize_system_variables()
    logging()

    symbol_info = base.session.get_instruments_info(category='perp', symbol=perp_name)
    qtyStepCoin = float(symbol_info['result']['list'][0]['priceFilter']['tickSize'])  #min coin incrementr
    coin_precision_info = symbol_info['result']['list'][0]['lotSizeFilter']['basePrecision']

    prev_attempt = 0
    #-----------------------------------------------------------------------------------------------------------------------------
    # WEBSOCKET
    ws = base.WebSocket(
    testnet=False,
    channel_type="linear")
    # api_key = local_settings.ake,
    # api_secret = local_settings.ase)
    ws.ticker_stream(symbol=perp_name,callback=handle_message_wrapper(logger))
    while True: #required
        if ws.is_connected() == False:
            if not handle_disconnection():
                break  # If unable to reconnect, exit the loop
        else:
            base.time.sleep(1)        
        #----------------------------------------------------------------------------------------------------------------------
        # Write price data to file every 20 seconds
        write_time = 10
        now = int(base.time.time())
        if time_index < now+write_time:
            time_index = now + 2*write_time
            # print(f'Rewriting file at:{time_index} \n')
            min_list_len = min(len(list(wsMarksPs.keys())),len(list(wsMarksPs.values())),len(list(wsIndexPs.values())))
            ws_df = base.pd.DataFrame.from_dict({"datetime":list(wsMarksPs.keys())[0:min_list_len],
                                                "markPrices":list(wsMarksPs.values())[0:min_list_len],
                                                "indexPrices":list(wsIndexPs.values())[0:min_list_len]})
            # print(f'len ws_df:{len(ws_df)}')
            ws_df.to_feather(ws_df_path)
        #----------------------------------------------------------------------------------------------------------------------
        # Write params to file after each attempt
        if params['attempt_num'] == prev_attempt+1:
            prev_attempt = prev_attempt + 1
            with open(f'{params_filePath}/{startParamsFileName}', 'w') as file:
                base.json.dump(params, file,indent=4)
        #----------------------------------------------------------------------------------------------------------------------
except KeyboardInterrupt:
    OnKeyboardInterrupt()
except Exception as e:
    # Log the exception
    base.logging.error(f"WebSocket encountered exception: {e}")
    print(f'\nErrored!')
    if ws.is_connected():
        print("Closing Connection due to exception!!!!")
        ws.exit()
#-----------------------------------------------------------------------------------------------------------------------------